In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Load data
df = pd.read_csv("../results/preprocessed_data.csv", parse_dates=True, index_col=0)

# Scale the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['TSLA']])

# Create sequences for LSTM
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i : i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 10
X, y = create_sequences(df_scaled, seq_length)

# Train-test split
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM Model
model = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, activation="relu"),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1)

# Make Predictions
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)

# Save Predictions
predictions_df = pd.DataFrame({"Actual": scaler.inverse_transform(y_test.reshape(-1, 1)).flatten(),
                               "Predicted": y_pred_rescaled.flatten()})
predictions_df.to_csv("../results/lstm_predictions.csv", index=False)

# Plot Results
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(predictions_df.index, predictions_df["Actual"], label="Actual", color="blue")
plt.plot(predictions_df.index, predictions_df["Predicted"], label="LSTM Prediction", color="red", linestyle="dashed")
plt.legend()
plt.title("LSTM Model - Actual vs Predicted TSLA Prices")
plt.xlabel("Time Step")
plt.ylabel("Stock Price")
plt.grid()
plt.show()
